In [3]:
from wholeslidedata.annotation.wholeslideannotation import WholeSlideAnnotation
from wholeslidedata.image.wholeslideimage import WholeSlideImage
from wholeslidedata.annotation.utils import plot_annotations
from matplotlib import pyplot as plt
from pprint import pprint
from wholeslidedata.annotation import utils as annotation_utils
import numpy as np
from PIL import Image, ImageDraw, ImageFont

# import pandas as pd
import sys
import os
import shutil
sys.path.append("C:/Users/mbotros/ASAP2.0/bin/")
from wholeslidedata.accessories.asap.imagewriter import WholeSlideMaskWriter

In [ ]:
def write_mask(wsi, wsa, spacing, tile_size=1024, suffix="_biopsy_mask_color.tif"):
    shape = wsi.shapes[wsi.get_level_from_spacing(spacing)]
    ratio = wsi.get_downsampling_from_spacing(spacing)
    write_spacing = wsi.get_real_spacing(spacing)

    mask_output_path = str(wsa.path).replace(".xml", suffix)
    
    wsm_writer = WholeSlideMaskWriter()
    wsm_writer.write(
        path=mask_output_path,
        spacing=write_spacing,
        dimensions=(shape[0], shape[1]),
        tile_shape=(tile_size, tile_size),
    )
    
    # make an image same size as the WSI
    mask = Image.new(mode='P', size=shape)
   
    # declare font
    font = ImageFont.truetype("arial.ttf", size=1500)
    
    # draw rectangle for every biopsy-outlines annotation
    for i, annotation in enumerate(wsa.annotations):
        
        bottom_left, top_right = tuple(annotation.coordinates[0]), tuple(annotation.coordinates[2])
        ImageDraw.Draw(mask).rectangle(xy=[bottom_left, top_right], width=175, outline=i + 1)
        # ImageDraw.Draw(mask).text(xy=bottom_left, text=" {}".format(i + 1), font=font, fill=1)
    
    # make it a numpy array
    mask = np.asarray(mask).T
    print('Shape mask: {}'.format(mask.shape))   
        
    # write output 
    for y_pos in range(0, shape[1], tile_size):
        for x_pos in range(0, shape[0], tile_size):
            mask_tile = mask[x_pos: x_pos + tile_size, y_pos : y_pos + tile_size].T
            if np.any(mask_tile):
                wsm_writer.write_tile(tile=mask_tile, coordinates=(int(x_pos), int(y_pos)))

    print("closing...")
    wsm_writer.save()
    print("done")

In [ ]:
# open df
df = pd.read_csv('C:/Users/mbotros/PhD/docs/dyplasia_cases_rbe_reader_study.csv')
case_list = df['case'].tolist()
print(case_list)

In [ ]:
# input_folder = 'L:/basic/divg/PATH-COMPAI/datasets/Barrett/CLASSIFIED/RBE/'
input_folder = 'L:/basic/divg/PATH-COMPAI/datasets/Barrett/CLASSIFIED/ASL HE + p53/HE/'
# input_folder = 'L:/basic/divg/PATH-COMPAI/datasets/Barrett/CLASSIFIED/LANS-Gland/'

image_files = sorted([os.path.join(input_folder, f) for f in os.listdir(input_folder) if 'tif' in f and 'biopsy' not in f])
annotation_files = sorted([os.path.join(input_folder, f) for f in os.listdir(input_folder) if 'xml' in f])

print(len(image_files))
print(len(annotation_files))

In [ ]:
output_folder = 'C:/Users/mbotros/PhD/data/reader_study_cases/'

In [ ]:
# define wsi and wsa (only take the biopsy outlines)
for path_to_wsi, path_to_wsa in zip(image_files, annotation_files):
    
    include = any(case in path_to_wsi for case in case_list)
    
    if include:
        
        # copy the wsi and wsa
        wsi_dest = os.path.join(output_folder, path_to_wsi.split('/')[-1])[:-1]
        print(wsi_dest)
        wsa_dest = os.path.join(output_folder, path_to_wsa.split('/')[-1])
        shutil.copy2(path_to_wsi, wsi_dest)
        shutil.copy2(path_to_wsa, wsa_dest)
        
        print('Image: {}'.format(wsi_dest))
        print('Anno: {}'.format(wsa_dest))

        wsi = WholeSlideImage(wsi_dest, backend='asap')
        wsa = WholeSlideAnnotation(wsa_dest, labels=['Biopsy-Outlines'])

        write_mask(wsi=wsi, wsa=wsa, spacing=0.25)
        print('\n')
    break